<a href="https://colab.research.google.com/github/Walbyq/prolongation-coefficient-analysis/blob/main/prolongation_coefficient_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np

from tabulate import tabulate
def tabulate_beauty(data):
    return print(tabulate(data, headers='keys', tablefmt='outline'))

# Загрузка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
prolongations_df = pd.read_csv('/content/drive/MyDrive/data/prolongations/prolongations.csv')

financial_df = pd.read_csv('/content/drive/MyDrive/data/prolongations/financial_data.csv')
financial_df = financial_df.drop('Account', axis=1)

# Анализ данных

## Таблица последних месяцев пролонгаций

In [ ]:
prolongations_df.head()

,id,month,AM
0,42,ноябрь 2022,Васильев Артем Александрович
1,453,ноябрь 2022,Васильев Артем Александрович
2,548,ноябрь 2022,Михайлов Андрей Сергеевич
3,87,ноябрь 2022,Соколова Анастасия Викторовна
4,429,ноябрь 2022,Соколова Анастасия Викторовна


Проверим значения столбцов 'AM' и 'month' таблицы 'prolongations_df' на аномальность:

In [ ]:
print(prolongations_df['AM'].value_counts(), "\n")
print(prolongations_df['month'].value_counts())

AM
Васильев Артем Александрович     139
Соколова Анастасия Викторовна     86
Попова Екатерина Николаевна       76
Смирнова Ольга Владимировна       69
Иванова Мария Сергеевна           48
Михайлов Андрей Сергеевич         31
Кузнецов Михаил Иванович          23
без А/М                            2
Федорова Марина Васильевна         2
Петрова Анна Дмитриевна            1
Name: count, dtype: int64 

month
декабрь 2023     69
декабрь 2022     67
сентябрь 2023    42
ноябрь 2023      40
март 2023        33
июнь 2023        31
февраль 2023     28
апрель 2023      28
август 2023      26
ноябрь 2022      24
октябрь 2023     24
июль 2023        23
май 2023         22
январь 2023      20
Name: count, dtype: int64


Видим, что в столбце 'AM' присутствет значение 'без А/М'. Проекты с неизвестным аккаунт-менеджером нам не интересны, так как мы анализируем работу сотрудников. Удалим эти строки:

In [ ]:
prolongations_df = prolongations_df[prolongations_df['AM'] != 'без А/М']

## Таблица отгрузок

In [ ]:
financial_df.head()

,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024
0,42,NaN,"36 220,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,657,первая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,657,вторая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,594,NaN,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,665,NaN,"10 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Приведем названия столбцов к нижнему регистру для соответствия с таблицей 'prolongations_df'
financial_df.columns = financial_df.columns.str.lower()

# Определим месяца
months = financial_df.columns[2:].to_list()

In [ ]:
# Заменим значения 'в ноль' на числа, а 'end' на 'стоп'
financial_df[months] = financial_df[months].replace('в ноль', float(0))
financial_df[months] = financial_df[months].replace('end', 'стоп')

Приведем числа к типу float, тк сейчас они имеют тип object:

In [ ]:
# Функция преобразования чисел из str в float
def convert_to_number(x):
    if isinstance(x, str) and x != 'стоп':
            return float(x.replace(' ', '').replace('\xa0', '').replace(',', '.'))
    else:
        return x

# Применение функции к всем столбцам месяцев
for month in months:
    financial_df.loc[:, month] = financial_df[month].apply(convert_to_number)

financial_df.head()

,id,причина дубля,ноябрь 2022,декабрь 2022,январь 2023,февраль 2023,март 2023,апрель 2023,май 2023,июнь 2023,июль 2023,август 2023,сентябрь 2023,октябрь 2023,ноябрь 2023,декабрь 2023,январь 2024,февраль 2024
0,42,NaN,36220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,657,первая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,657,вторая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,594,NaN,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,665,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Теперь проверим значения столбца 'причина дубля' таблицы 'financial_df':

In [ ]:
print(financial_df['причина дубля'].value_counts())

причина дубля
первая часть оплаты    114
вторая часть оплаты     99
доп работы              38
основные работы         38
изменение ЮЛ            11
карты, банки             1
Name: count, dtype: int64


Видим, что строк со значением 'первая часть оплаты' больше, чем строк со значением 'вторая часть оплаты'.

Рассмотрим все проекты, у которых есть только одно из необходимых 2-х частей отгрузок:

In [ ]:
first_pay_ids = set(financial_df['id'].loc[financial_df['причина дубля'] == 'первая часть оплаты'])
second_pay_ids = set(financial_df['id'].loc[financial_df['причина дубля'] == 'вторая часть оплаты'])

main_work_ids = set(financial_df['id'].loc[financial_df['причина дубля'] == 'основные работы'])
other_work_ids = set(financial_df['id'].loc[financial_df['причина дубля'] == 'доп работы'])

# Вывод строк
print("Количество проектов, с отгрузкой только за:")

print("первую часть оплаты:", len(first_pay_ids - second_pay_ids))
print("вторую часть оплаты:", len(second_pay_ids - first_pay_ids))

print("основные работы:", len(main_work_ids - other_work_ids))
print("доп работы:", len(other_work_ids - main_work_ids))

Количество проектов, с отгрузкой только за:
первую часть оплаты: 22
вторую часть оплаты: 1
основные работы: 8
доп работы: 6


Причины по которым это может происходить могут быть разными:

*   Данные пропущены при сборе;
*   Данные были разделены на части, но вся оплата происходила сразу;
*   Ошибки при вводе.

Недостающих данных достаточно много, поэтому их удаление может исказить результаты, важные при оценке работы сотрудников. Будем работать с ними как с цельными платежами:

In [ ]:
# Удаление столбца 'причина дубля'
financial_df = financial_df.drop('причина дубля', axis=1)

# Функция агрегации
def aggregate(x):
    if x.isna().all():
        return np.nan
    elif ('стоп' in x.values):
        return 'стоп'
    else:
        try:
            return np.nansum(x)
        except ValueError:
            return np.nan

# Агрегация по 'id'
financial_df = financial_df.groupby('id').agg(aggregate)

In [ ]:
# Замена нулевых значений на значения предыдущего месяца
for i in range(len(months) - 1):
    current_month = months[i]
    prev_month = months[i - 1]

    financial_df[current_month] = financial_df.apply(
        lambda row: row[prev_month] if row[current_month] == 0 else row[current_month],
        axis=1
    )

# Рассчеты

In [ ]:
# Объединение датафреймов по 'id'
merged_df = pd.merge(prolongations_df, financial_df, on='id', how='inner')

In [ ]:
prolong_month_df = prolongations_df.groupby('id')['month'].apply(lambda x: list(x))

# Создаём словарь для быстрого доступа к спискам месяцев
months_dict = prolong_month_df.to_dict()

In [ ]:
def check_stop(row):
    month = row['month']
    id = row['id']

    # Получаем список всех последних месяцев реализации, соответствующих id
    id_months = months_dict.get(id, [])

    # Находим порядковые номера текущего и предыдущего месяцев реализации
    month_idx = id_months.index(month)
    month_serial = months.index(month)

    if month_idx > 0:
        prev_month = id_months[month_idx - 1]
        prev_month_serial = months.index(prev_month)
    else:
        prev_month_serial = 0

    # Находим порядковые номера месяцев, в которые проект завершился досрочно
    stop_months_serial = [months.index(month) for month in months if row[month] == 'стоп']

    # Проверяем, останавливался ли проект досрочно перед завершением текущего договора, но после предыдущего
    if prev_month_serial == 0:
        was_stop = any(prev_month_serial <= stop <= month_serial for stop in stop_months_serial)
    else:
        was_stop = any(prev_month_serial < stop <= month_serial for stop in stop_months_serial)

    return was_stop

# Фильтрация строк со значением 'стоп'
stop_df = merged_df[(merged_df[months] == 'стоп').any(axis=1)]

# Находим 'id' проектов, которые заврешались досрочно
stop_df['has_stop'] = stop_df.apply(check_stop, axis=1)
indexes_to_remove = stop_df[stop_df['has_stop']].index

# Очищаем найденные 'id' из таблицы
merged_df = merged_df.drop(indexes_to_remove)

# Остальные 'стоп' заменим на nan
merged_df[months] = merged_df[months].replace('стоп', np.nan)

<ipython-input-1424-930467ff2c6d>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stop_df['has_stop'] = stop_df.apply(check_stop, axis=1)
<ipython-input-1424-930467ff2c6d>:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df[months] = merged_df[months].replace('стоп', np.nan)


In [ ]:
# Функция для агрегации по именам и месяцам
def get_coef(df, months):
    managers = df['AM'].unique()
    results = []

    for manager in managers:
        manager_df = df[df['AM'] == manager]

        for i in range(1, len(months)):
            last_last_month = months[i - 2]
            last_month = months[i - 1]
            current_month = months[i]

            # Суммы отгрузки проектов пролонгированных в первый месяц после завершения
            total_sum = manager_df[last_month][manager_df['month'] == last_month].sum()

            # Сумма отгрузки последнего месяца реализации всех завершившихся в прошлом месяце проектов
            notnan_month = manager_df[manager_df[current_month].notna()]
            notnan_month_sum = notnan_month[last_month][notnan_month['month'] == last_month].sum()

            if i >= 2:
                # Сумма отгрузки проектов, пролонгированных во второй месяц
                nan_last_month = manager_df[manager_df[last_month].isna()]
                nan_last_month_sum = nan_last_month[last_last_month][nan_last_month['month'] == last_last_month].sum()

                # Сумма отгрузки последнего месяца проектов, не пролонгированных в первый
                nan_last_notnan_month = nan_last_month[nan_last_month[current_month].notna()]
                nan_last_notnan_month_sum = nan_last_notnan_month[last_last_month][notnan_month['month'] == last_last_month].sum()

                k2 = nan_last_notnan_month_sum / nan_last_month_sum if nan_last_month_sum != 0 else np.nan

            else:
                k2 = np.nan

            k1 = notnan_month_sum / total_sum if total_sum != 0 else np.nan

            results.append({
                "AM": manager,
                "month": current_month,
                "year": int(current_month[-4:]),
                "k1": k1,
                "k2": k2
            })

    return pd.DataFrame(results)

# Агрегация по именам и месяцам
coef_df = get_coef(merged_df, months)

In [ ]:
aggregated_month_df = coef_df.groupby(['AM', 'month']).agg({
    'k1': 'mean',
    'k2': 'mean'})

print('Агрегация по сотрудникам и месяцам:')
aggregated_month_df

Агрегация по сотрудникам и месяцам:


k1        k2
AM                           month                            
Васильев Артем Александрович август 2023    0.577042  0.000000
                             апрель 2023    0.372375  0.110203
                             декабрь 2022   0.582674       NaN
                             декабрь 2023   0.359416  0.545715
                             июль 2023      0.448260  0.000000
...                                              ...       ...
Федорова Марина Васильевна   сентябрь 2023       NaN       NaN
                             февраль 2023        NaN       NaN
                             февраль 2024        NaN  0.000000
                             январь 2023         NaN       NaN
                             январь 2024    0.000000       NaN

[135 rows x 2 columns]

In [ ]:
aggregated_year_df = coef_df.groupby(['AM', 'year']).agg({
    'k1': 'mean',
    'k2': 'mean'})

print('Агрегация по сотрудникам и годам:')
aggregated_year_df

Агрегация по сотрудникам и годам:


k1        k2
AM                            year                    
Васильев Артем Александрович  2022  0.582674       NaN
                              2023  0.519192  0.101371
                              2024  0.564875  0.089013
Иванова Мария Сергеевна       2022  0.000000       NaN
                              2023  0.570349  0.000000
                              2024       NaN       NaN
Кузнецов Михаил Иванович      2022       NaN       NaN
                              2023  0.626645  0.000000
                              2024  0.666691  0.000000
Михайлов Андрей Сергеевич     2022  0.952252       NaN
                              2023  0.626441  0.000000
                              2024  0.000000  0.500000
Петрова Анна Дмитриевна       2022       NaN       NaN
                              2023  1.000000       NaN
                              2024       NaN       NaN
Попова Екатерина Николаевна   2022  0.400087       NaN
                              2023  0.463699  0.094553
                              2024  0.536647  0.500000
Смирнова Ольга Владимировна   2022       NaN       NaN
                              2023  0.501426  0.363335
                              2024  0.468789  0.203905
Соколова Анастасия Викторовна 2022  0.518368       NaN
                              2023  0.565773  0.071604
                              2024  0.431679  0.000000
Федорова Марина Васильевна    2022       NaN       NaN
                              2023       NaN       NaN
                              2024  0.000000  0.000000

In [ ]:
aggregated_depart_month_df = coef_df.groupby('month').agg({
    'k1': 'mean',
    'k2': 'mean'})

print('Агрегация по месяцам:')
aggregated_depart_month_df

Агрегация по месяцам:


,k1,k2
month,,
август 2023,0.598100,0.200000
апрель 2023,0.538537,0.067529
декабрь 2022,0.490676,NaN
декабрь 2023,0.537779,0.233444
июль 2023,0.417464,0.086436
июнь 2023,0.161393,0.018988
май 2023,0.768724,0.000000
март 2023,0.693167,0.340992
ноябрь 2023,0.526329,0.029602


In [ ]:
aggregated_depart_year_df = coef_df.groupby('year').agg({
    'k1': 'mean',
    'k2': 'mean'})

print('Агрегация по годам:')
aggregated_depart_year_df

Агрегация по годам:


,k1,k2
year,,
2022,0.490676,NaN
2023,0.553166,0.105657
2024,0.381240,0.198911
